In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.under_sampling import EditedNearestNeighbours
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import tensorflow as tf
import pickle
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dot
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K
import keras
import random
import numpy as np
import os
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.under_sampling import EditedNearestNeighbours
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import tensorflow as tf
import pickle
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dot
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K
import keras
import random
import numpy as np
import os
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelEncoder


# Define directories
train_dir = '/kaggle/input/plantdoc/PlantDoc-Dataset/train' 
test_dir = '/kaggle/input/plantdoc/PlantDoc-Dataset/test'    

# Parameters
img_height, img_width = 64, 64  
batch_size = 32
num_classes = 27  


def load_images_from_directory(directory):
    images = []
    labels = []
    classes = os.listdir(directory)
    for label in classes:
        class_dir = os.path.join(directory, label)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                img = load_img(img_path, target_size=(img_height, img_width))
                img_array = img_to_array(img)
                images.append(img_array)
                labels.append(label)
    return np.array(images), np.array(labels)

x_train, y_train = load_images_from_directory(train_dir)
x_test, y_test = load_images_from_directory(test_dir)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_encoded = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_encoded = to_categorical(y_test_encoded, num_classes=num_classes)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train_encoded, test_size=0.2, stratify=y_train)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

x_train_aug = []
y_train_aug = []
for i in range(len(x_train)):
    aug_iter = datagen.flow(x_train[i:i+1], y_train[i:i+1], batch_size=1)
    x_aug, y_aug = next(aug_iter)
    x_train_aug.append(x_aug[0])
    y_train_aug.append(y_aug[0])

x_train_aug = np.array(x_train_aug)
y_train_aug = np.array(y_train_aug)

x_train_flat = x_train_aug.reshape((x_train_aug.shape[0], -1))  
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_flat, y_train_aug)

x_train_resampled = x_train_resampled.reshape(-1, img_height, img_width, 3)

train_generator = datagen.flow(x_train_resampled, y_train_resampled, batch_size=batch_size)

In [17]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, InceptionV3, MobileNetV2, EfficientNetB0, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

num_classes = 27

input_shape = (64, 64, 3)

def create_resnet50_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model)  
    model.add(GlobalAveragePooling2D())  
    model.add(Dense(1024, activation='relu'))  
    model.add(Dense(num_classes, activation='softmax'))  

    return model

resnet50_model = create_resnet50_model(input_shape, num_classes)
resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_mobilenetv2_model(input_shape, num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model)  
    model.add(GlobalAveragePooling2D())  
    model.add(Dense(1024, activation='relu')) 
    model.add(Dense(num_classes, activation='softmax'))

    return model

mobilenetv2_model = create_mobilenetv2_model(input_shape, num_classes)
mobilenetv2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_efficientnetb0_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model)  
    model.add(GlobalAveragePooling2D())  
    model.add(Dense(1024, activation='relu'))  
    model.add(Dense(num_classes, activation='softmax'))  

    return model

efficientnetb0_model = create_efficientnetb0_model(input_shape, num_classes)
efficientnetb0_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_vgg16_model(input_shape, num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model)  
    model.add(GlobalAveragePooling2D())  
    model.add(Dense(1024, activation='relu'))  
    model.add(Dense(num_classes, activation='softmax'))  

    return model

vgg16_model = create_vgg16_model(input_shape, num_classes)
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/tmp/ipykernel_37/559834048.py:29: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)


In [14]:
history1 = resnet50_model.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = resnet_model.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100
  1/122 ━━━━━━━━━━━━━━━━━━━━ 1:35:43 47s/step - accuracy: 0.0000e+00 - loss: 3.9779

I0000 00:00:1725955982.582500     113 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_28', 20 bytes spill stores, 20 bytes spill loads



122/122 ━━━━━━━━━━━━━━━━━━━━ 73s 208ms/step - accuracy: 0.0932 - loss: 3.5147 - val_accuracy: 0.0754 - val_loss: 18.0809
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.2530 - loss: 2.6524 - val_accuracy: 0.0690 - val_loss: 63.9065
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.2952 - loss: 2.4111 - val_accuracy: 0.0927 - val_loss: 6.1883
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.3367 - loss: 2.2652 - val_accuracy: 0.1121 - val_loss: 4.6924
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.3777 - loss: 2.0977 - val_accuracy: 0.1401 - val_loss: 4.8668
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.4224 - loss: 1.9280 - val_accuracy: 0.1250 - val_loss: 4.9690
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.4711 - loss: 1.7464 - val_accuracy: 0.1595 - val_loss: 4.2542
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.4899 - loss: 1.6987 - val_accu

In [20]:
history2 = mobilenetv2_model.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = mobilenetv2_model.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 11s 86ms/step - accuracy: 0.1776 - loss: 2.9378 - val_accuracy: 0.0647 - val_loss: 5.1614
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.2551 - loss: 2.6160 - val_accuracy: 0.0625 - val_loss: 5.0643
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.3414 - loss: 2.2823 - val_accuracy: 0.1250 - val_loss: 3.7224
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.3783 - loss: 2.1096 - val_accuracy: 0.0776 - val_loss: 6.0263
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.4041 - loss: 2.0400 - val_accuracy: 0.0560 - val_loss: 6.0312
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.4075 - loss: 1.9320 - val_accuracy: 0.1056 - val_loss: 5.0931
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.4631 - loss: 1.7786 - val_accuracy: 0.0388 - val_loss: 5.7036
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.4716 - loss: 1.7308 

In [21]:
history3 = efficientnetb0_model.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = efficientnetb0_model.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 124s 418ms/step - accuracy: 0.1645 - loss: 3.0224 - val_accuracy: 0.2004 - val_loss: 3.1326
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.3958 - loss: 2.1157 - val_accuracy: 0.1961 - val_loss: 3.4478
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.5171 - loss: 1.6223 - val_accuracy: 0.2177 - val_loss: 3.4283
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.5923 - loss: 1.3174 - val_accuracy: 0.2522 - val_loss: 3.2805
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.6608 - loss: 1.1043 - val_accuracy: 0.2522 - val_loss: 3.4011
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.6857 - loss: 1.0168 - val_accuracy: 0.2672 - val_loss: 3.6802
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.7413 - loss: 0.8372 - val_accuracy: 0.2134 - val_loss: 4.3046
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.7474 - loss: 0.8272

In [22]:
history4 = vgg16_model.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = vgg16_model.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 24s 107ms/step - accuracy: 0.0388 - loss: 7.2183 - val_accuracy: 0.0409 - val_loss: 3.2963
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.0370 - loss: 3.2964 - val_accuracy: 0.0409 - val_loss: 3.2965
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.0306 - loss: 3.2962 - val_accuracy: 0.0237 - val_loss: 3.2961
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.0361 - loss: 3.2963 - val_accuracy: 0.0431 - val_loss: 3.2957
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.0320 - loss: 3.2961 - val_accuracy: 0.0237 - val_loss: 3.2960
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.0346 - loss: 3.2961 - val_accuracy: 0.0453 - val_loss: 3.2959
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.0326 - loss: 3.2961 - val_accuracy: 0.0237 - val_loss: 3.2963
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.0303 - loss: 3.2960 

In [27]:
from tensorflow.keras.applications import Xception,DenseNet121,VGG19,ResNet101,EfficientNetV2B0,MobileNetV3Small


def create_DenseNet121_model(input_shape, num_classes):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model) 
    model.add(GlobalAveragePooling2D())  
    model.add(Dense(1024, activation='relu'))  
    model.add(Dense(num_classes, activation='softmax'))  

    return model

Densenet_model = create_DenseNet121_model(input_shape, num_classes)
Densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


def create_vgg19_model(input_shape, num_classes):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model)  
    model.add(GlobalAveragePooling2D())  
    model.add(Dense(1024, activation='relu'))  
    model.add(Dense(num_classes, activation='softmax')) 

    return model

vgg19_model = create_vgg19_model(input_shape, num_classes)
vgg19_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_resnet101_model(input_shape, num_classes):
    base_model = ResNet101(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model)  
    model.add(GlobalAveragePooling2D())  
    model.add(Dense(1024, activation='relu'))  
    model.add(Dense(num_classes, activation='softmax'))  

    return model

resnet101_model = create_resnet101_model(input_shape, num_classes)
resnet101_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_efficientnetv2b0_model(input_shape, num_classes):
    base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model)  
    model.add(GlobalAveragePooling2D()) 
    model.add(Dense(1024, activation='relu'))  
    model.add(Dense(num_classes, activation='softmax'))  

    return model

efficientnetv2b0_model = create_efficientnetv2b0_model(input_shape, num_classes)
efficientnetv2b0_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_mobilenetv3_model(input_shape, num_classes):
    base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model)  
    model.add(GlobalAveragePooling2D())  
    model.add(Dense(1024, activation='relu')) 
    model.add(Dense(num_classes, activation='softmax'))  

    return model

mobilenetv3_model = create_mobilenetv3_model(input_shape, num_classes)
mobilenetv3_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/opt/conda/lib/python3.10/site-packages/keras/src/applications/mobilenet_v3.py:449: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [28]:
def create_simple_cnn(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

simple_cnn = create_simple_cnn(input_shape, num_classes)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
history5 = simple_cnn.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = simple_cnn.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 11s 69ms/step - accuracy: 0.0465 - loss: 16.5617 - val_accuracy: 0.0474 - val_loss: 3.2532
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.0465 - loss: 3.2633 - val_accuracy: 0.0216 - val_loss: 3.2977
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.0390 - loss: 3.2947 - val_accuracy: 0.0259 - val_loss: 3.2902
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.0403 - loss: 3.2941 - val_accuracy: 0.0560 - val_loss: 3.2329
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.0430 - loss: 3.2855 - val_accuracy: 0.0323 - val_loss: 3.2863
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.0438 - loss: 3.2913 - val_accuracy: 0.0302 - val_loss: 3.2879
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.0436 - loss: 3.2873 - val_accuracy: 0.0409 - val_loss: 3.2846
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.0475 - loss: 3.2877 

In [30]:
history7 = Densenet_model.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = Densenet_model.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 181s 573ms/step - accuracy: 0.0906 - loss: 3.4974 - val_accuracy: 0.0603 - val_loss: 4.9516
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.1870 - loss: 2.7899 - val_accuracy: 0.1703 - val_loss: 3.7490
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.2504 - loss: 2.5101 - val_accuracy: 0.1379 - val_loss: 3.4867
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.2917 - loss: 2.3707 - val_accuracy: 0.1466 - val_loss: 4.1146
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.3135 - loss: 2.3360 - val_accuracy: 0.1185 - val_loss: 5.5377
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.3499 - loss: 2.1764 - val_accuracy: 0.2091 - val_loss: 4.1675
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.4208 - loss: 1.9408 - val_accuracy: 0.1315 - val_loss: 154.1733
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.4356 - loss: 1.85

In [31]:
history8 = resnet101_model.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = resnet101_model.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100


I0000 00:00:1725961606.991129     113 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_54', 40 bytes spill stores, 40 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_52', 40 bytes spill stores, 40 bytes spill loads



122/122 ━━━━━━━━━━━━━━━━━━━━ 142s 401ms/step - accuracy: 0.1109 - loss: 3.4853 - val_accuracy: 0.0474 - val_loss: 87.6521
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - accuracy: 0.2117 - loss: 2.7038 - val_accuracy: 0.1444 - val_loss: 4.4446
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - accuracy: 0.2765 - loss: 2.4850 - val_accuracy: 0.0453 - val_loss: 18041.2012
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - accuracy: 0.2756 - loss: 2.4877 - val_accuracy: 0.0797 - val_loss: 74.8945
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 11s 89ms/step - accuracy: 0.3323 - loss: 2.2505 - val_accuracy: 0.1228 - val_loss: 4.9717
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.3730 - loss: 2.0991 - val_accuracy: 0.0647 - val_loss: 1382.1005
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.3879 - loss: 2.0076 - val_accuracy: 0.1724 - val_loss: 24.1358
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 11s 89ms/step - accuracy: 0.4275 - loss: 1

In [32]:
history10 = vgg19_model.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = vgg19_model.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step - accuracy: 0.0310 - loss: 6.2765 - val_accuracy: 0.0366 - val_loss: 3.2965
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - accuracy: 0.0401 - loss: 3.2964 - val_accuracy: 0.0409 - val_loss: 3.2964
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - accuracy: 0.0288 - loss: 3.2963 - val_accuracy: 0.0409 - val_loss: 3.2960
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - accuracy: 0.0396 - loss: 3.2960 - val_accuracy: 0.0323 - val_loss: 3.2956
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - accuracy: 0.0362 - loss: 3.2961 - val_accuracy: 0.0539 - val_loss: 3.2955
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - accuracy: 0.0351 - loss: 3.2959 - val_accuracy: 0.0539 - val_loss: 3.2954
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - accuracy: 0.0323 - loss: 3.2960 - val_accuracy: 0.0539 - val_loss: 3.2955
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - accuracy: 0.0347 - loss: 3.2961 -

In [33]:
history11 = efficientnetv2b0_model.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = efficientnetv2b0_model.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100
  1/122 ━━━━━━━━━━━━━━━━━━━━ 3:06:45 93s/step - accuracy: 0.0312 - loss: 3.2989

I0000 00:00:1725963701.728731     114 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_13', 20 bytes spill stores, 20 bytes spill loads



122/122 ━━━━━━━━━━━━━━━━━━━━ 155s 514ms/step - accuracy: 0.1711 - loss: 2.9383 - val_accuracy: 0.2134 - val_loss: 2.9462
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.3945 - loss: 2.0841 - val_accuracy: 0.2155 - val_loss: 3.1263
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.5051 - loss: 1.6213 - val_accuracy: 0.2134 - val_loss: 3.6846
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.5891 - loss: 1.3429 - val_accuracy: 0.2328 - val_loss: 3.3864
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.6517 - loss: 1.1294 - val_accuracy: 0.2371 - val_loss: 3.7363
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.6961 - loss: 0.9718 - val_accuracy: 0.2112 - val_loss: 3.9199
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.7356 - loss: 0.8630 - val_accuracy: 0.2198 - val_loss: 3.9075
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.7491 - loss: 0.7828 - val_accu

In [34]:
history12 = mobilenetv3_model.fit(train_generator, epochs=100, validation_data=(x_val, y_val))
loss, accuracy = mobilenetv3_model.evaluate(x_test,y_test_encoded)
print(f"Test accuracy: {accuracy}")

Epoch 1/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 83s 380ms/step - accuracy: 0.1172 - loss: 3.2439 - val_accuracy: 0.0862 - val_loss: 3.7983
Epoch 2/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.2951 - loss: 2.4098 - val_accuracy: 0.1185 - val_loss: 3.3367
Epoch 3/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.4095 - loss: 2.0133 - val_accuracy: 0.1444 - val_loss: 3.7234
Epoch 4/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.4965 - loss: 1.6792 - val_accuracy: 0.1078 - val_loss: 5.5389
Epoch 5/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.5553 - loss: 1.4858 - val_accuracy: 0.1336 - val_loss: 4.6666
Epoch 6/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.6243 - loss: 1.2473 - val_accuracy: 0.1401 - val_loss: 4.1631
Epoch 7/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.6751 - loss: 1.1010 - val_accuracy: 0.1315 - val_loss: 6.3327
Epoch 8/100
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.6778 - loss: 1.0366 